In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [ ]:

!pip install trl

In [ ]:

!pip install --upgrade trl peft


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.4.7
    Uninstalling trl-0.4.7:
      Successfully uninstalled trl-0.4.7
  Attempting uninstall: peft
    Found existing installation: peft 0.4.0
    Uninstalling peft-0.4.0:
      Successfully uninstalled peft-0.4.0


In [ ]:
!pip install datasets

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
use_auth_token=True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "eleldar/language-detection"

# # The instruction dataset to use
# dataset_name = "sirgecko/jpvnlearning"

# Fine-tuned model name
new_model = "sirgecko/languade-detection_3"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 128

# Batch size per GPU for evaluation
per_device_eval_batch_size = 128

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 128

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
from os import error
import pandas as pd

train_path = "/content/drive/MyDrive/data_language_detection/train (7).csv"
test_path = "/content/drive/MyDrive/data_language_detection/test (4).csv"

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)


In [ ]:
df_train.head()

,label,text
0,swedish,Det som vi gör här är förmodligen mycket vikti...
1,lithuanian,"Manau, kad jeigu norime, kad mūsų piliečiai la..."
2,greek,Η συμπερίληψη των θαλάσσιων συνόρων στο κεφάλα...
3,polish,"Chcielibyśmy, aby ta stocznia dokonała realnej..."
4,slovene,Vendar so se takšni pozitivni obeti hitro izni...


In [ ]:
label2id = {value: index for index, value in enumerate(df_train["label"].unique())}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
len(label2id)

22

In [ ]:
from datasets import Dataset

def df_to_arrow(df):
  return Dataset.from_pandas(df).shuffle(seed=20)

In [ ]:
train_dataset = df_to_arrow(df_train)
test_dataset = df_to_arrow(df_test)

In [ ]:
!apt-get update
!apt-get install -y nvidia-driver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [872 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,083 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ub

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


In [ ]:
!nvidia-smi

Wed May 22 15:08:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # thêm padding để các câu có độ dài bằng nhau
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type='CLASSIFICATION',
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
from datasets import load_metric
import numpy as np

In [ ]:
!nvidia-smi

Wed May 22 15:08:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-21-82e51f34e7fd>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

In [ ]:
train_dataset

Dataset({
    features: ['labels', 'text'],
    num_rows: 219999
})

In [ ]:
# Tokenizer dataset và thêm padding
# Tại sao lại phải tokenizer riêng?
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=max_seq_length,return_tensors="pt" )
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns='text')
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns='text')

Map:   0%|          | 0/219999 [00:00<?, ? examples/s]

Map:   0%|          | 0/44000 [00:00<?, ? examples/s]

In [ ]:
!nvidia-smi

Wed May 22 15:09:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
def encode_labels(example):
    """Map string labels to integers."""
    example["labels"] = label2id[example["labels"]]
    return example
train_dataset = train_dataset.map(encode_labels, batched=False)
test_dataset = test_dataset.map(encode_labels, batched=False)

Map:   0%|          | 0/219999 [00:00<?, ? examples/s]

Map:   0%|          | 0/44000 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 219999
})

In [ ]:
!nvidia-smi

Wed May 22 15:09:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=22,
    ignore_mismatched_sizes=True
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at eleldar/language-detection and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([20, 768]) in the checkpoint and torch.Size([22, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([20]) in the checkpoint and torch.Size([22]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!nvidia-smi

Wed May 22 15:11:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from peft import get_peft_model
model = get_peft_model(model, peft_config)

In [ ]:
!nvidia-smi

Wed May 22 15:11:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from transformers import Trainer

In [ ]:
!nvidia-smi

Wed May 22 15:11:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir= "output",    # thư mục lưu model
    num_train_epochs=num_train_epochs, #số lần lặp
    per_device_train_batch_size = per_device_train_batch_size, #độ lớn mỗi batch
    per_device_eval_batch_size = per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps, #độ tích lũy gradient descent trước khi cập nhập trọng số
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps, # số bước mà sau đó các thông số về lost, accuracy được trả về
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length, # nhóm các dữ liệu với nhau dựa trên độ dài
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    remove_unused_columns=False,
)

# Set supervised fine-tuning parameters

trainer = Trainer(
    model=model,
    # dataset_text_field="text",
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    args=training_arguments,
    compute_metrics=compute_metrics
)

In [ ]:
!nvidia-smi

Wed May 22 15:11:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0              25W /  70W |   1227MiB / 15360MiB |     31%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Train model
trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,3.114900
50,3.017400
75,2.837700
100,2.684500
125,2.563600
150,2.420300
175,2.307000
200,2.223500
225,2.133400
250,1.985800


KeyboardInterrupt: 

In [ ]:
new_model = "trained model"

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
tokenizer.save_pretrained(new_model)

('trained model/tokenizer_config.json',
 'trained model/special_tokens_map.json',
 'trained model/sentencepiece.bpe.model',
 'trained model/added_tokens.json',
 'trained model/tokenizer.json')

In [ ]:
print(id2label)

{0: 'swedish', 1: 'lithuanian', 2: 'greek', 3: 'polish', 4: 'slovene', 5: 'hungarian', 6: 'dutch', 7: 'bulgarian', 8: 'italian', 9: 'german', 10: 'danish', 11: 'finnish', 12: 'portoguese', 13: 'slovak', 14: 'spanish', 15: 'romanian', 16: 'czech', 17: 'estonian', 18: 'french', 19: 'english', 20: 'latvian', 21: 'japanese'}


In [ ]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=22,
    ignore_mismatched_sizes=True
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at eleldar/language-detection and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([20, 768]) in the checkpoint and torch.Size([22, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([20]) in the checkpoint and torch.Size([22]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
peft_model = PeftModel.from_pretrained(base_model,
                                      '/content/lora_model',
                                      is_trainable=False) ## is_trainable mean just a forward pass jsut to get a sumamry

In [ ]:
base_model = AutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=len(label2id),
            label2id=label2id,
            id2label=id2label,
            ignore_mismatched_sizes=True)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at eleldar/language-detection and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([20, 768]) in the checkpoint and torch.Size([22, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([20]) in the checkpoint and torch.Size([22]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import re
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(
    model=peft_model,
    return_all_scores=True,
    tokenizer=tokenizer,
    device=0)



def get_result(text_list):
  predict = pipe(text_list)
  result = []
 # for i in predict:
  #  language_num = re.findall(r'\d+', i['label'])
    #result.append([int(language_num[0])])


  return predict

print(get_result([ '愛してる']))


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
The model 'PeftModel' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceCl

[[{'label': 'swedish', 'score': 0.047123804688453674}, {'label': 'lithuanian', 'score': 0.036654360592365265}, {'label': 'greek', 'score': 0.0425727441906929}, {'label': 'polish', 'score': 0.07907731086015701}, {'label': 'slovene', 'score': 0.04968016967177391}, {'label': 'hungarian', 'score': 0.04477124288678169}, {'label': 'dutch', 'score': 0.04571111500263214}, {'label': 'bulgarian', 'score': 0.04014863818883896}, {'label': 'italian', 'score': 0.048433318734169006}, {'label': 'german', 'score': 0.04854670912027359}, {'label': 'danish', 'score': 0.0449908971786499}, {'label': 'finnish', 'score': 0.05028786510229111}, {'label': 'portoguese', 'score': 0.03295347839593887}, {'label': 'slovak', 'score': 0.04718682914972305}, {'label': 'spanish', 'score': 0.060312941670417786}, {'label': 'romanian', 'score': 0.054986536502838135}, {'label': 'czech', 'score': 0.03960516303777695}, {'label': 'estonian', 'score': 0.038393594324588776}, {'label': 'french', 'score': 0.04526768997311592}, {'lab

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [ ]:
model.push_to_hub("sirgecko/language_detection_tuesdaynight", check_pr=True)
tokenizer.push_to_hub("sirgecko/language_detection_tuesdaynight",check_pr=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/9.46M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sirgecko/language_detection_tuesdaynight/commit/206a4ed0a429a3c9e27ce127ca57d1d58c2ea869', commit_message='Upload tokenizer', commit_description='', oid='206a4ed0a429a3c9e27ce127ca57d1d58c2ea869', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Empty VRAM
#del model
#del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model_final = PeftModel.from_pretrained(base_model, new_model)
model_final = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
print(id2label)

{0: 'swedish', 1: 'lithuanian', 2: 'greek', 3: 'polish', 4: 'slovene', 5: 'hungarian', 6: 'dutch', 7: 'bulgarian', 8: 'italian', 9: 'german', 10: 'danish', 11: 'finnish', 12: 'portoguese', 13: 'slovak', 14: 'spanish', 15: 'romanian', 16: 'czech', 17: 'estonian', 18: 'french', 19: 'english', 20: 'latvian', 21: 'japanese'}


In [ ]:
from os import error
import pandas as pd

train_path = "/content/drive/MyDrive/data_language detection/train (7).csv"
test_path = "/content/drive/MyDrive/data_language detection/test (4).csv"

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)


In [ ]:
label2id = {value: index for index, value in enumerate(df_train["label"].unique())}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
import re
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(
    model=model_final,
    tokenizer=tokenizer,
    device=0)



def get_result(text_list):
  predict = pipe(text_list)
  result = []
  for i in predict:
    language_num = re.findall(r'\d+', i['label'])
    result.append([int(language_num[0])])
  return result

print(get_result([ 'I love you.', 'Je taime.','Te quiero','Ich liebe dich.','Ti amo.','愛してる','Eu te amo.','Σ αγαπώ ','Ik hou van jou.']))

NameError: name 'model_final' is not defined

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [ ]:
model_final.push_to_hub("sirgecko/language_detection_tuesdaylatenight", check_pr=True)

tokenizer.push_to_hub("sirgecko/language_detection_tuesdaylatenight",check_pr=True)


pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sirgecko/language_detection_tuesdaylatenight/commit/418635aa52543a913130f4a9022f6f8a1e07fef4', commit_message='Upload tokenizer', commit_description='', oid='418635aa52543a913130f4a9022f6f8a1e07fef4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("saved_model")
tokenizer = AutoTokenizer.from_pretrained("saved_model")